In [1]:
import baseline_methods as bm
import numpy as np
import pandas as pd
%load_ext autoreload
%autoreload 2

In [3]:
us_delay = np.load('./udata/udelay.npy')
#shape[0]-airports shape[1]-timestamp shape[2]-arrival/delay
#us_delay_1 arrival delay 
us_delay_1 = us_delay[:,:,0]
#us_delay_1 departure delay 
us_delay_2 = us_delay[:,:,1]

In [4]:
us_delay_1.shape

(70, 78912)

# HA

In [26]:
y_predict,y_test = bm.historical_average_predict(us_delay_1,period=2*18,test_ratio=0.2)
ha_a_mae,ha_a_rmse,ha_a_r2 =bm.test_error(y_predict,y_test)
y_predict,y_test = bm.historical_average_predict(us_delay_2,period=2*18,test_ratio=0.2)
ha_d_mae,ha_d_rmse,ha_d_r2 = bm.test_error(y_predict,y_test)
print(f"The test error for arrival prediction delay using HA is {ha_a_mae,ha_a_rmse,ha_a_r2}")
print(f"The test error for delay prediction delay using HA is {ha_d_mae,ha_d_rmse,ha_d_r2}")

d:\杨蕙菡\20230310\STPN\baseline_methods.py:58: RuntimeWarning: Mean of empty slice.
  y_predict[k, i - n_train] = historical[k, :][~np.isnan(historical[k, :])].mean()
c:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


The test error for arrival prediction delay using HA is (9.08868575737624, 11.847158267257146, 0.04013236239198659)
The test error for delay prediction delay using HA is (6.518559479523361, 8.631168654546585, 0.06914510610205782)


# VAR

In [6]:
time = [2,5,11]

In [7]:
y_predict_a,y_test_a = bm.var_predict(us_delay_1,test_ratio=0.2)
y_predict_d,y_test_d = bm.var_predict(us_delay_2,test_ratio=0.2)
var_a_mae = []
var_a_rmse = []
var_a_r2 = []
var_d_mae = []
var_d_rmse = []
var_d_r2 = []
for i in time:
    a,b,c =bm.test_error(y_predict_a[i,:,:].T,y_test_a)
    var_a_mae.append(a)
    var_a_rmse.append(b)
    var_a_r2.append(c)
    a,b,c =bm.test_error(y_predict_d[i,:,:].T,y_test_d)
    var_d_mae.append(a)
    var_d_rmse.append(b)
    var_d_r2.append(c)

print(f"The test error for arrival prediction delay using Var in 1.5 hour, 3 hour and 6 hour is {var_a_mae,var_a_rmse,var_a_r2}")
print(f"The test error for delay prediction delay using Var in 1.5 hour, 3 hour and 6 hour is is {var_d_mae,var_d_rmse,var_d_r2}")

var 146.21613575693726
var 80.03026012831663
var 146.21613575693726
var 80.03026012831663
var 146.21613575693726
var 80.03026012831663
The test error for arrival prediction delay using Var in 1.5 hour, 3 hour and 6 hour is ([7.795143750162598, 8.122843663922692, 8.479481960176464], [10.467598702219648, 10.823749555638715, 11.237335232953244], [0.250625643856055, 0.19876452871152483, 0.13636273805176735])
The test error for delay prediction delay using Var in 1.5 hour, 3 hour and 6 hour is is ([5.561195565149635, 5.8165432121599725, 6.164752850760663], [7.655895249704866, 7.925223562827234, 8.303570831678394], [0.2676178737832824, 0.2151822520620722, 0.13845977201435722])


# ARIMA

In [37]:
y_predict_d,y_test_d=bm.arima_predict(us_delay_2,test_ratio=0.2)

In [42]:
var_a_mae = []
var_a_rmse = []
var_a_r2 = []
var_d_mae = []
var_d_rmse = []
var_d_r2 = []
for i in time:
    a,b,c =bm.test_error(y_predict_d[i,:,:].T,y_test_d)
    var_d_mae.append(a)
    var_d_rmse.append(b)
    var_d_r2.append(c)

print(f"The test error for arrival prediction delay using Var in 1.5 hour, 3 hour and 6 hour is {var_a_mae,var_a_rmse,var_a_r2}")
print(f"The test error for delay prediction delay using Var in 1.5 hour, 3 hour and 6 hour is is {var_d_mae,var_d_rmse,var_d_r2}")

The test error for arrival prediction delay using Var in 1.5 hour, 3 hour and 6 hour is ([], [], [])
The test error for delay prediction delay using Var in 1.5 hour, 3 hour and 6 hour is is ([9.766172855159779, 9.760533847625258, 9.771212764326236], [12.173278606686496, 12.129616073491928, 12.182781077574033], [0.042873992903643576, 0.0497276272019489, 0.0413791430577799])


# SVM

In [9]:
a,b = us_delay_2.shape
us_delay_2_h = np.zeros((a, int(b/4)))

# 迭代每一行
for i in range(a):
    # 迭代每个时间戳（不包括最后一个时间戳）
    for j in range(0, b-1, 4):
        # 计算相邻时间戳的总延误时间
        total_delay = us_delay_2[i, j] + us_delay_2[i, j+1] + us_delay_2[i, j+2]+ us_delay_2[i, j+3]#+ us_delay_2[i, j+4]+ us_delay_2[i, j+5]+ us_delay_2[i, j+6]+ us_delay_2[i, j+7]
        # 将结果存储在新数组中的相应位置
        us_delay_2_h[i, j//4] = total_delay

In [ ]:
y_predict_d,y_test_d=bm.var_predict_svr(us_delay_2_h,test_ratio=0.2)

In [11]:
time = [0,1,2]
var_a_mae = []
var_a_rmse = []
var_a_r2 = []
var_d_mae = []
var_d_rmse = []
var_d_r2 = []
for i in time:
    a,b,c =bm.test_error(y_predict_d[i,:,:].T,y_test_d)
    var_d_mae.append(a)
    var_d_rmse.append(b)
    var_d_r2.append(c)

print(f"The test error for arrival prediction delay using Var in 2 hour, 4 hour and 6 hour is {var_a_mae,var_a_rmse,var_a_r2}")
print(f"The test error for delay prediction delay using Var in 2 hour, 4 hour and 6 hour is is {var_d_mae,var_d_rmse,var_d_r2}")

The test error for arrival prediction delay using Var in 1.5 hour, 3 hour and 6 hour is ([], [], [])
The test error for delay prediction delay using Var in 1.5 hour, 3 hour and 6 hour is is ([18.1733441078865, 21.71026009962508, 19.19220219180221], [23.290944613739796, 27.694911217715514, 24.63388518099744], [-59.878386935875874, -85.07734953730801, -67.1011988147711])


# GAT

to run the GAT model, please turn to the `training_u_gat.py`, type command in the terminal with `python training_u_gat.py`.


# LSTM